In [10]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
import seaborn as sns
import os
# widely used for a variety of image and video analysis
import cv2
import random
import glob

In [8]:
df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/food2%-food1%/day4-n10-food2%/2024-04-30_14-31-44_td5.000_2024-04-30_14-31-44_td5.analysis.csv')

VIDEO OF LARVAE 

In [13]:

# Set the image size
image_size = 1400

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter('/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/food2%-food1%/day4-n10-food2%/output_video_test.mp4', fourcc, 10.0, (image_size, image_size))

df = df.sort_values('frame_idx')

def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Dictionary to store the last index of each track
last_index = {}
# Dictionary to maintain color assignment for each track
track_colors = {}

# Iterate over each frame
for frame in range(df['frame_idx'].min(), df['frame_idx'].max() + 1):
    frame_df = df[df['frame_idx'] == frame]

    # Initialize a blank image for this frame
    frame_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

    # Determine which tracks are currently active
    current_tracks = set(frame_df['track'].unique())

    # Identify tracks that have disappeared
    disappeared_tracks = set(last_index.keys()) - current_tracks
    for track in disappeared_tracks:
        del last_index[track]  # Remove disappeared tracks from last_index

    # Update the track_colors dictionary for any new tracks
    for track in current_tracks:
        if track not in track_colors:
            track_colors[track] = generate_random_color()

    for track in current_tracks:
        track_df = frame_df[frame_df['track'] == track]
        if track_df.empty:
            continue

        current_index = track_df.index[0]

        if track in last_index:
            # Retrieve the previous frame's data using the stored last index
            prev_index = last_index[track]
            prev_track_df = df.loc[prev_index]

            for part in ['head', 'body', 'tail']:
                prev_x = prev_track_df[f'{part}.x']
                prev_y = prev_track_df[f'{part}.y']
                curr_x = track_df[f'{part}.x'].values[0]
                curr_y = track_df[f'{part}.y'].values[0]

                # Check if any of the coordinates are NaN before drawing
                if not np.isnan([prev_x, prev_y, curr_x, curr_y]).any():
                    # Convert to integers
                    prev_x, prev_y, curr_x, curr_y = map(int, [prev_x, prev_y, curr_x, curr_y])
                    # Draw the line with the corresponding track color
                    cv2.line(frame_image, (prev_x, prev_y), (curr_x, curr_y),
                             color=track_colors[track], thickness=4)

        # Update the last index for this track
        last_index[track] = current_index

    # Write the frame to the video output
    video_output.write(frame_image)

# Release the video writer
video_output.release()




VIDEO OF TRACKS

In [15]:
# initially read the csv file

df = pd.read_csv('/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/food2%-food1%/day4-n10-food2%/2024-04-30_14-31-44_td5.000_2024-04-30_14-31-44_td5.analysis.csv')

# definition to generate a random RGB color (3 channels)
   # return a tuple with three integers ranging from 0 to 255
def generate_random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

# Set the image size
   # current resolution of videos 1400 1400
image_size = 1400

# Directory and output settings
output_path = '/Volumes/lab-windingm/home/users/cochral/AttractionRig/data/sleap-csv-2024-04-30/food2%-food1%/day4-n10-food2%/output__test.mp4'

# Define the codec and create VideoWriter object
# This object is used to write frames to the output video at 10 frames per second with the defined image size
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec used to compress the frames
video_output = cv2.VideoWriter(output_path, fourcc, 10.0, (image_size, image_size))

# Assume 'df' is already loaded and sorted by 'frame_idx'
df = df.sort_values('frame_idx')

# Dictionary to store the last index of each track
  # Stores the last index processed for each track to link the current frame with the previous one
last_index = {}
# Dictionary to maintain unique color assignment for each track
track_colors = {}

# Initialize a cumulative image
  # Initializes a zero-valued image array (cumulative_image) that will be used to draw the paths of the tracked objects
  # dont get this as much 
cumulative_image = np.zeros((image_size, image_size, 3), dtype=np.uint8)

# Iterate over each frame
for frame in range(df['frame_idx'].min(), df['frame_idx'].max() + 1):
    # ensure frame is current frame 
    frame_df = df[df['frame_idx'] == frame]

    # Update the track_colors dictionary for any new tracks in the current frame
    for track in frame_df['track'].unique():
        if track not in track_colors:
            track_colors[track] = generate_random_color()

    # iterate over each track

    for track in frame_df['track'].unique():
        # ensure the track is the current track iteration
        track_df = frame_df[frame_df['track'] == track]
        # if there is no data for the track in the current frame it skips the rest of the loop
        if track_df.empty:
            continue

        # Stores the current index of the track data
           # This helps in linking positions between consecutive frames.

        current_index = track_df.index[0]

        # If the track was present in the previous frame (checked using last_index), it retrieves the previous coordinates and draws lines from the previous positions to the current positions for each body part (head, body, tail)

        if track in last_index:
            # Retrieve the previous frame's data using the stored last index
            prev_index = last_index[track]
            prev_track_df = df.loc[prev_index]

            # iterate over each part 
            for part in ['head', 'body', 'tail']:
                prev_x = prev_track_df[f'{part}.x']
                prev_y = prev_track_df[f'{part}.y']
                curr_x = track_df[f'{part}.x'].values[0]
                curr_y = track_df[f'{part}.y'].values[0]

                # Check if any of the coordinates are NaN before drawing
        
                if not np.isnan([prev_x, prev_y, curr_x, curr_y]).any():
                    # Converts coordinates to integers
                    prev_x, prev_y, curr_x, curr_y = map(int, [prev_x, prev_y, curr_x, curr_y])
                    # Draw the line on cumulative_image with the corresponding track color
                    cv2.line(cumulative_image, (prev_x, prev_y), (curr_x, curr_y),
                             color=track_colors[track], thickness=1)

        # Update the last index for this track
          # important for next frame processing 
        last_index[track] = current_index

    # Write the cumulative image to the video output
    video_output.write(cumulative_image)

# Release the video writer
video_output.release()


FIRST SCRIPT

Initializes a new frame_image for each frame, which means only the current movements are drawn on this image. The image is reset to black (empty) at the beginning of processing each frame. This approach produces a video where each frame only shows the movements for that specific frame, without retaining the history of previous movements


SECOND SCRIPT

Utilizes a single cumulative_image that persists throughout all frames, accumulating all tracking lines drawn from the start to the end of the video. This method creates a video where each frame shows all previous movements overlayed, forming a continuous trace of each track's path

